In [1]:
import sys 
  
# appending a path 
sys.path.append('/home/ubuntu/duc.nm195858/Controllable-Multi-Objective-Reranking') 

import os
import pickle as pkl
import random
from collections import defaultdict
import numpy as np
import math
import json
from librerank.utils import save_file

2023-11-06 09:07:19.535564: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-06 09:07:19.604529: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-06 09:07:19.604580: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-06 09:07:19.604623: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-06 09:07:19.616596: I tensorflow/core/platform/cpu_feature_g

In [2]:
raw_sample = '/home/ubuntu/duc.nm195858/Controllable-Multi-Objective-Reranking-v2/Data/ad/raw_data/raw_sample.csv'

In [9]:
user_tl = defaultdict(list)
behav_tl = defaultdict(list)
item_sft = {}
item_dft = {}
user_prof = {}
last_ts = {}
uid_set = set()
iid_set = set()
idx = 0
pos = 0
First = True
with open(raw_sample, 'r') as r:
        for line in r:
            if First:
                First = False
                continue
            # user ID, timestamp, adgroup_id, ... , clk: 1 for click, 0 for not click
            uid, ts, iid, pid, noclk, clk = line.split(',')
            d = float(ts)
            pos += int(clk)
            user_tl[int(uid)].append([d, int(iid), int(clk)])  #{user_1: [[timestamp_1, iid_2, 0], [timestamp, iid, 1]]}
            idx += 1
print('num of records: ', idx, 'pos vs neg: ', pos / (idx - pos), 'user:', len(user_tl))

num of records:  26557961 pos vs neg:  0.05422599045209166 user: 1141729


In [12]:
user_tl[2]

[[1494250392.0, 258454, 0],
 [1494250392.0, 19024, 0],
 [1494250392.0, 508794, 1]]

In [15]:

removed_1, removed_2 = 0, 0
pos = 0
total_len, max_len, min_len = 0, 0, 1e9
for uid in user_tl.keys():
    tl = user_tl[uid]
    # Filter out users with fewer than 3 records.
    if len(tl) > 3:
        tl = sorted(tl, key=lambda k: k[0], reverse=True)  #sort by timestamp
        tl_len = len(tl)
        tl = np.array(tl)
        if sum(tl[:, 2]) == 0:
            removed_2 += 1
            continue

        # dentify and remove consecutive records with no clicks in a time window of 5 minutes.
        si, ei = 0, 1
        while si < tl_len:
            while ei < tl_len and (tl[si][0] - tl[ei][0] <= 60 * 5):
                ei += 1
            if sum(tl[si:ei][:, 2]) == 0:
                si = ei
                ei += 1
            else:
                break

        if si >= tl_len:
            print('wrong', si, ei, tl[:, 2])
            exit()
        total_len += (ei - si)
        max_len = max(max_len, ei-si)
        min_len = min(min_len, ei-si)

        uid_set.add(uid)
        remain_tl = tl[si: ei][-30:]

        pos += sum(remain_tl[:, 2])
        remain_tl = remain_tl.tolist()
        remain_tl.reverse()
        user_tl[uid] = remain_tl
        last_ts[uid] = remain_tl[0][0]
        for v in remain_tl:
            iid_set.add(v[1])
    else:
        removed_1 += 1
print('filter <= 4: num of user:', len(uid_set), ' num of item: ', len(iid_set), ' removed <= 3: ', removed_1,
        'remove all 0', removed_2, 'pos per list: ', pos / len(uid_set))
print('max len:', max_len, 'min len:', min_len, 'average len:', total_len / len(uid_set))


filter <= 4: num of user: 430846  num of item:  343437  removed <= 3:  379001 remove all 0 331883 pos per list:  1.1659780989030883
max len: 81 min len: 1 average len: 5.593527617756693


In [17]:
user_tl[2]

[[1494250392.0, 258454, 0],
 [1494250392.0, 19024, 0],
 [1494250392.0, 508794, 1]]

In [20]:
user_tl = defaultdict(list)
# behav_tl = defaultdict(list)
item_sft = {}
item_dft = {}
user_prof = {}
last_ts = {}
uid_set = set()
iid_set = set()
idx = 0
pos = 0

First = True
with open(raw_sample, 'r') as r:
    for line in r:
        if First:
            First = False
            continue
        uid, ts, iid, pid, noclk, clk = line.split(',')
        d = float(ts)
        pos += int(clk)
        user_tl[int(uid)].append([d, int(iid), int(clk)])
        idx += 1
print('num of records: ', idx, 'pos vs neg: ', pos / (idx - pos), 'user:', len(user_tl))

removed_1, removed_2 = 0, 0
pos = 0
total_len, max_len, min_len = 0, 0, 1e9
for uid in user_tl.keys():
    tl = user_tl[uid]
    if len(tl) > 2:
        tl = sorted(tl, key=lambda k: k[0], reverse=True)
        tl_len = len(tl)
        tl = np.array(tl)
        if sum(tl[:, 2]) == 0:
            removed_2 += 1
            continue

        si, ei = 0, 1
        while si < tl_len:
            while ei < tl_len and (tl[si][0] - tl[ei][0] <= 60 * 5):
                ei += 1
            if sum(tl[si:ei][:, 2]) == 0:
                si = ei
                ei += 1
            else:
                break

        if si >= tl_len:
            print('wrong', si, ei, tl[:, 2])
            exit()
        total_len += (ei - si)
        max_len = max(max_len, ei-si)
        min_len = min(min_len, ei-si)

        uid_set.add(uid)
        remain_tl = tl[si: ei][-30:]

        pos += sum(remain_tl[:, 2])
        remain_tl = remain_tl.tolist()
        remain_tl.reverse()
        user_tl[uid] = remain_tl
        last_ts[uid] = remain_tl[0][0]
        for v in remain_tl:
            iid_set.add(v[1])
    else:
        removed_1 += 1
print('filter < 3: num of user:', len(uid_set), ' num of item: ', len(iid_set), ' removed <= 2: ', removed_1,
        'remove all 0', removed_2, 'pos per list: ', pos / len(uid_set))
print('max len:', max_len, 'min len:', min_len, 'average len:', total_len / len(uid_set))

num of records:  26557961 pos vs neg:  0.05422599045209166 user: 1141729
filter < 3: num of user: 483049  num of item:  349404  removed <= 2:  80813 remove all 0 577867 pos per list:  1.163730801637101
max len: 81 min len: 1 average len: 5.311055400176794


In [22]:
user_tl[2]

[[1494250392.0, 508794.0, 1.0],
 [1494250392.0, 19024.0, 0.0],
 [1494250392.0, 258454.0, 0.0]]

In [23]:
user_profile = '/home/ubuntu/duc.nm195858/Controllable-Multi-Objective-Reranking-v2/Data/ad/raw_data/user_profile.csv'
ad_feature = '/home/ubuntu/duc.nm195858/Controllable-Multi-Objective-Reranking-v2/Data/ad/raw_data/ad_feature.csv'

In [27]:
First = True
with open(user_profile, 'r') as r:
    for line in r:
        if First:
            First = False
            continue
        values = line.split(',')
        # print(values)
        uid = int(values[0])
        if uid in uid_set:
            user_prof[uid] = values[1:]
print('finish loading user profile, num profile:', len(user_prof))
# uid_set = set(user_prof.keys())

First = True
with open(ad_feature, 'r') as r:
    for line in r:
        if First:
            First = False
            continue
        iid, cate_id, cam_id, cust_id, brand, price = line.split(',')
        iid = int(iid)
        if iid in iid_set:
            item_sft[iid] = [iid, cate_id, cam_id, cust_id, brand]
            item_dft[iid] = [eval(price)]
print('finish loading ad feature, num item:', len(item_sft))


profile_fnum, itm_dens_fnum, itm_spar_fnum= 8, 1, 5
print('profile fnum:', profile_fnum, 'itm sparse fnum:', itm_spar_fnum, 'itm dense fnum:', itm_dens_fnum)
wo_profile = 0
for uid in uid_set:
    if not user_prof.__contains__(uid):
        user_prof[uid] = ['null' for _ in range(profile_fnum)]
        wo_profile += 1
print('user w/o profile:', wo_profile)

finish loading user profile, num profile: 452881
finish loading ad feature, num item: 349404
profile fnum: 8 itm sparse fnum: 5 itm dense fnum: 1
user w/o profile: 30168


In [29]:
def convert(data, ft_map):
    res = []
    for i, v in enumerate(data):
        res.append(ft_map[i][v])
    return res

In [30]:
uid_map, usr_ft_map, item_ft_map = {}, [{} for _ in range(profile_fnum)], [{} for _ in range(itm_spar_fnum)]
ft_idx = 1
for iid in iid_set:
    itm_ft = item_sft[iid]
    for i in range(itm_spar_fnum):
        if not item_ft_map[i].__contains__(itm_ft[i]):
            item_ft_map[i][itm_ft[i]] = ft_idx
            ft_idx += 1
for uid in uid_set:
    prof = user_prof[uid]
    for i in range(profile_fnum):
        if not usr_ft_map[i].__contains__(prof[i]):
            usr_ft_map[i][prof[i]] = ft_idx
            ft_idx += 1

print('total feature', ft_idx)
print('user profile')
for i in range(profile_fnum):
    print(len(usr_ft_map[i]), end='  ')
print()
print('item sparse feature')
for i in range(itm_spar_fnum):
    print(len(item_ft_map[i]), end='  ')
print()


# map old feature id to new one
res = []
for uid in uid_set:
    ft_spar, ft_dens, label, profile = [], [], [], []
    for d, iid, clk in user_tl[uid]:
        ft_spar.append(convert(item_sft[iid], item_ft_map))
        ft_dens.append(item_dft[iid])
        label.append(clk)
    profile.append(convert(user_prof[uid], usr_ft_map))

    res.append([uid, profile, ft_spar, ft_dens, label])

cate_num = len(item_ft_map[1])
user_tl, item_sft, item_dft, item_ft_map, usr_ft_map = [], [], [], [], []

# split train:va;:test = 4:5:1
random.shuffle(res)
num = len(res)
train_list, val_list, test_list = res[: int(0.4 * num)], res[int(0.4 * num): int(0.9 * num)], res[int(0.9*num):]

total feature 815998
user profile
98  14  3  8  5  4  3  6  
item sparse feature
349404  5525  237392  156439  67096  


In [1]:
import tensorflow as tf

matrix = tf.constant([[1, 2, 3],
                     [4, 5, 6],
                     [7, 8, 9]])
sum_along_axis_0 = tf.reduce_sum(matrix, axis=0)
print(sum_along_axis_0.numpy())



2023-11-16 14:59:09.714362: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 14:59:09.756233: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 14:59:09.756271: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 14:59:09.756293: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 14:59:09.763137: I tensorflow/core/platform/cpu_feature_g

[12 15 18]


2023-11-16 14:59:13.488329: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 146 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:1a:00.0, compute capability: 8.6
2023-11-16 14:59:13.488957: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 146 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:3d:00.0, compute capability: 8.6
2023-11-16 14:59:13.489531: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1886] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 22282 MB memory:  -> device: 2, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:89:00.0, compute capability: 8.6


In [32]:
train_list[0]

[258328,
 [[815857, 815871, 815867, 815872, 815861, 815862, 815863, 815877]],
 [[795364, 50451, 744468, 555634, 10],
  [653849, 50451, 653850, 601190, 10],
  [647506, 10517, 647507, 513129, 285587],
  [316344, 1195, 316345, 316346, 196527],
  [707918, 50451, 707919, 506913, 121135],
  [601188, 50451, 601189, 601190, 10]],
 [[288.0], [286.0], [298.0], [179.0], [398.0], [238.0]],
 [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]]

In [3]:
import torch
loss = torch.nn.CrossEntropyLoss()
input = torch.randn(3, 5, requires_grad=True)
target = torch.empty(3, dtype=torch.long).random_(5)
output = loss(input, target)
# >>> output.backward()

In [6]:
target

tensor([1, 4, 2])

In [5]:
input

tensor([[ 1.6922, -1.0358,  0.0269, -0.2551, -0.0267],
        [ 0.6068, -1.2827,  0.4204,  0.2397,  1.9741],
        [ 0.2439,  1.8989, -0.2013,  0.2739, -1.2664]], requires_grad=True)

In [9]:
import numpy as np

# Giả sử loss_numpy có hình dạng (a, b)
loss_numpy = np.array([[1, 2], [3, 4]])

# Lấy batch size
batch_size = 32

# Thêm chiều mới cho batch size
loss_numpy = loss_numpy[np.newaxis, :, :]
n_samples, n_mo_obj, n_mo_sol = loss_numpy.shape

# Mở rộng chiều mới cho batch size
# loss_numpy = np.tile(loss_numpy, [batch_size, 1, 1])

print(loss_numpy.shape)


(1, 2, 2)


In [11]:
n_samples

1